### NYU CDS

### Fall 2021

### Introduction to Data Science

### Project 2

### student netid: 

### deadline: Dec 06, 2021, 11:59pm

In [2]:
# Name: Ziwei Xu (Vanessa)
# student netid: zx657

---
# Data analysis Project 2
### Correlation and Regression of Movie Ratings Data
---

In [4]:
import re
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso

### Dataset description

This dataset features ratings data of 400 movies from 1097 research participants. 

* 1st row: Headers (Movie titles/questions) – note that the indexing in this list is from 1
* Row 2-1098: Responses from individual participants
* Columns 1-400: These columns contain the ratings for the 400 movies (0 to 4, and missing)
* Columns 401-421: These columns contain self-assessments on sensation seeking behaviors (1-5)
* Columns 422-464: These columns contain responses to personality questions (1-5)
* Columns 465-474: These columns contain self-reported movie experience ratings (1-5)
* Column 475: Gender identity (1 = female, 2 = male, 3 = self-described)
* Column 476: Only child (1 = yes, 0 = no, -1 = no response)
* Column 477: Movies are best enjoyed alone (1 = yes, 0 = no, -1 = no response)

Note that we did most of the data munging for you already (e.g. Python interprets commas in a csv file as separators, so we removed all commas from movie titles), but you still need to handle missing data.

In [181]:
# load the dataset, and skip the header/first row
data = np.genfromtxt('movieReplicationSet2.csv', delimiter = ',', skip_header = 1)
# read the csv file as a dataframe in pandas
df = pd.read_csv('movieReplicationSet2.csv')
# get a subset of the dataset for just the movie ratings
movies = df.copy()
movies = movies.iloc[:,0:400]
# size of the movies data set - 1097 rows × 400 columns




### Q1:


**Note:** For all missing values in the data, use the average of the corresponding column so to fill in the missing data. 



In this problem, under **the most correlated**, we consider the largest correlation in the absolute value.


1.1. For every user in the given data, find its most correlated user. 

1.2. What is the pair of the most correlated users in the data? 

1.3. What is the value of this highest correlation?

1.4. For users 0, 1, 2, \dots, 9, print their most correlated users. 



In [183]:
# Question 1.1

# first, fill missing values in the data with average of the corresponding column
for i in range(400):
    movies.iloc[:,i] = movies.iloc[:,i].fillna(movies.iloc[:,i].mean())
# initialize empty container to store each rho value
temp = np.empty([1097,2])
temp[:] = np.NaN # convert to NaN
# now, find the most correlated user using Spearman Rho
for i in range(1097): # loop through each user/row
    lst = [] # initialize empty list to store Spearman Rho for ith user with all other users
    for j in range(1097): # loop through each user apart from themselves
        if i == j:
            lst.append(0)
        else:
            r,p = stats.spearmanr(movies.iloc[i], movies.iloc[j]) # compute Spearman Rho
            lst.append(r) # store coefficient in list
    mostcorr = max(lst, key = abs)
    temp[i,0] = mostcorr # gives us the most correlated user's correlation coeffient
    temp[i,1] = lst.index(mostcorr) # gives us the most correlated user number
# turning the table into a pandas dataframe
#corr = pd.DataFrame(temp)
corr = pd.DataFrame(temp, columns =['correlation', 'matched_user'])
corr.head(5)

,correlation,matched_user
0,0.815985,314.0
1,0.929290,831.0
2,0.981346,896.0
3,0.761645,704.0
4,0.657508,784.0


In [184]:
# Question 1.2

# first of all, sort values of correlation dataframe by the 
# absolute values of correlation coefficients in descending order
corr.sort_values("correlation", ascending = False, key=abs, inplace = True)
# we can see that the top two rows are the most correlated pair 
# of users 896 and 831
corr

,correlation,matched_user
896,0.998442,831.0
831,0.998442,896.0
152,0.997129,896.0
219,0.992261,896.0
452,0.992206,896.0
...,...,...
804,-0.158983,1087.0
701,-0.153416,145.0
102,0.150736,755.0
912,0.147734,1018.0


In [195]:
# Question 1.3

# as we have already sorted the dataframe by correlation coefficient in descending order
# all we need to do in order to find the highest absolute value of correlation coeffient
# is to find the coefficient in the first row
corr.iloc[0,0]
# the value of this highest correlation coeffient is 0.9984418642391248

0.9984418642391248

In [196]:
# Question 1.4

# finding most correlated users for users 0, 1, 2 ... 9
for i in range(10):
    print(corr.loc[i][1])
# so for users 0-9, the most correlated users are respectively
# 314, 831, 896, 704, 784, 156, 818, 708, 821, 1004

314.0
831.0
896.0
704.0
784.0
156.0
818.0
708.0
821.0
1004.0


### Q2:

We want to find a model between the ratings and the personal part of the data. To do so, consider:


**Part 1**: the ratings of all users over columns 1-400: 

-- Columns 1-400: These columns contain the ratings for the 400 movies (0 to 4, and missing);

call this part `df_rate`


and 


**Part 2**:  the part of the data which includes all users over columns 401-474

-- Columns 401-421: These columns contain self-assessments on sensation seeking behaviors (1-5)

-- Columns 422-464: These columns contain responses to personality questions (1-5)

-- Columns 465-474: These columns contain self-reported movie experience ratings (1-5)

call this part `df_pers`.

---

Our main task is to model: 


`df_pers = function(df_rate)`


---

**Note:** Split the original data into training and testing as the ratio 0.80: 0.20. 


2.1. Model `df_pers = function(df_rate)` by using the linear regression. 

What are the errors on: (i) the training part; (ii) the testing part?




2.2. Model `df_pers = function(df_rate)` by using the ridge regression with hyperparamter values alpha from [0.0, 1e-8, 1e-5, 0.1, 1, 10]. 

For every of the previous values for alpha, what are the errors on: (i) the training part; (ii) the testing part?

What is a best choice for alpha?



2.3. Model `df_pers = function(df_rate)` by using the lasso regression with hyperparamter values alpha from [1e-3, 1e-2, 1e-1, 1]. 

For every of the previous values for alpha, what are the errors on: (i) the training part; (ii) the testing part?

What is a best choice for alpha?


**Note**: Ignore any `convergence warning` in case you may obtain in the Lasso regression.




In [197]:
# first off, divide the dataset into subsets described above
df_rate = df.copy()
df_rate = df_rate.iloc[:,0:400] # 1097 rows × 400 columns
# first, fill missing values in the data with average of the corresponding column means
for i in range(400):
    df_rate.iloc[:,i] = df_rate.iloc[:,i].fillna(df_rate.iloc[:,i].mean())
df_pers = df.copy()
df_pers = df_pers.iloc[:,400:474] # 1097 rows × 74 columns (should be 77)
for i in range(74):
    df_pers.iloc[:,i] = df_pers.iloc[:,i].fillna(df_pers.iloc[:,i].mean())

In [198]:
# Then, split the original data into training and testing as the ratio 0.80: 0.20
lst2 = []
for i in range(400):
    lst2.append(df_rate.iloc[:,i])
X = np.transpose(lst2)
y = df_pers
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# X_train - 877 rows × 400 columns
# X_test - 220 rows × 400 columns
# y_train - 877 rows × 74 columns
# y_test - 220 rows × 74 columns
#np.transpose([df_rate.iloc[:,0],df_rate.iloc[:,1],df_rate.iloc[:,2]])

In [199]:
# Question 2.1

# Model df_pers = function(df_rate) by using the linear regression
#X = np.transpose([data[:,0],data[:,1],data[:,2]]) # IQ, hours worked, years education
#Y = data[:,3] # income
regr = LinearRegression() # linearRegression function from linear_model
regr.fit(X_train,y_train) # use fit method 
rSqr = regr.score(X_train,y_train) # 0.498 - realistic
betas = regr.coef_ # m
yInt = regr.intercept_ # b

# training error 
train_error = mean_squared_error(y_train, regr.predict(X_train)) # 0.619629274630977

# testing error
test_error = mean_squared_error(y_test, regr.predict(X_test)) # 2.8122699539823657

In [200]:
# Question 2.2

# Model df_pers = function(df_rate) by using the ridge regression 
# with hyperparamter values alpha from [0.0, 1e-8, 1e-5, 0.1, 1, 10]
alphas = [0.0, 1e-8, 1e-5, 0.1, 1, 10] # setting alpha levels in a list    
for alpha in alphas:
    print(alpha)
    regr = Ridge(alpha = alpha)
    regr.fit(X_train,y_train) # use fit method 
    rSqr = regr.score(X_train,y_train) # 
    betas = regr.coef_ # m
    yInt = regr.intercept_ # b
    print(rSqr)
    # training error for each alpha level
    train_error = mean_squared_error(y_train, regr.predict(X_train)) # 
    print(train_error)
    # testing error for each alpha level
    test_error = mean_squared_error(y_test, regr.predict(X_test)) # 
    print(test_error)

# the best choice for alpha would be 10, as it produces the lowest level of
# testing error of 1.78 among all choices of alphas

0.0
0.5045629735802987
0.6090092380127824
3.630390365102066
1e-08
0.5045629735802987
0.6090092380127825
3.6303903519293237
1e-05
0.5045629735791909
0.6090092380141402
3.630377192480192
0.1
0.504466541198379
0.6091274125425393
3.509974215424232
1
0.5002278418115678
0.6143231675130144
2.917858334105748
10
0.4573268425803339
0.6669255986396038
1.9468094010950956


In [ ]:
# Question 2.3

# Model df_pers = function(df_rate) by using the lasso regression with 
# hyperparamter values alpha from [1e-3, 1e-2, 1e-1, 1]
alphas = [1e-3, 1e-2, 1e-1, 1]

for alpha in alphas:
    print(alpha)
    regr = Lasso(alpha = alpha)
    regr.fit(X_train,y_train) # use fit method 
    rSqr = regr.score(X_train,y_train)
    betas = regr.coef_ # m
    yInt = regr.intercept_ # b
    print(rSqr)
    # training error for each alpha level
    train_error = mean_squared_error(y_train, regr.predict(X_train)) # 
    print(train_error)
    # testing error for each alpha level
    test_error = mean_squared_error(y_test, regr.predict(X_test)) # 
    print(test_error)

# the best choice for alpha would be 0.1, as it produces the lowest level of
# testing error of 1.22 among all choices of alphas

0.001


/Users/vanessaxu/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.0869083744089494, tolerance: 0.13023621236229263
  model = cd_fast.enet_coordinate_descent(
/Users/vanessaxu/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.260558881454699, tolerance: 0.18457965728635883
  model = cd_fast.enet_coordinate_descent(
/Users/vanessaxu/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_coordinate_descent.py:530: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.06403516460221681, tolerance: 0.05403269033674381
  model = cd_fast.enet_coordinate_descent(
/Users/vanessaxu/opt/anaconda3/lib/python3.8/site-pa